In [1]:
# Chapter 6

In [5]:
# Classification with KNN

In [73]:
from sklearn.datasets import load_iris
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors

In [2]:
iris = load_iris()

In [3]:
X = iris.data
Y = iris.target

In [4]:
XTrain, XTest, YTrain, YTest =\
ms.train_test_split(X,Y, test_size = 0.3, random_state = 7)

In [5]:
# With KNN we need to find the appropriate value of k. and we can do this 
# with the help of GridSearchCV

In [19]:
k_neighbours = list(range(1,21,2)) # list of even numbers as candidates
n_grid = [{'n_neighbors':k_neighbours}]

In [20]:
model = neighbors.KNeighborsClassifier()

In [21]:
# We are performing cross-validation with 10 folds in our training data

In [25]:
cv_knn = GridSearchCV(estimator = model, param_grid =  n_grid,\
                     cv = ms.KFold(n_splits=10))

In [26]:
cv_knn.fit(XTrain, YTrain)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [27]:
best_k = cv_knn.best_params_['n_neighbors']

In [28]:
print('The best parameter is k = {0}'.format(best_k))

The best parameter is k = 11


In [31]:
# For visualisation purposes we will concentrate on only two features:
# sepal width and petal lenght

knnclf = neighbors.KNeighborsClassifier(n_neighbors=best_k)

knnclf.fit(XTrain[:, 2:4], YTrain) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')

In [32]:
y_pred = knnclf.predict(XTest[:, 2:4])

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(YTest, y_pred)

array([[12,  0,  0],
       [ 0, 14,  2],
       [ 0,  2, 15]])

In [35]:
# 4 instances have been missclassified. 

In [36]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(YTest, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        12
          1       0.88      0.88      0.88        16
          2       0.88      0.88      0.88        17

avg / total       0.91      0.91      0.91        45



In [1]:
# Classification with Logistic Regression

In [94]:
import pandas as pd

In [99]:
bc = pd.read_csv('../data/breast-cancer-wisconsin.csv')

In [101]:
bc['class'] = bc['class'].astype('category')
bc['class'].describe()

count     699
unique      2
top         2
freq      458
Name: class, dtype: int64

In [102]:
# Let's prepare our data by separating the labels from the rest of the dataset

In [103]:
X = bc.drop(['class'], axis=1)

In [104]:
X = X.values

In [105]:
Y_raw = bc['class'].values

In [106]:
# Let's change the labels to be 0 or 1 using LabelEncoder

In [107]:
from sklearn import preprocessing

In [108]:
label_enc = preprocessing.LabelEncoder()
label_enc.fit(Y_raw)

LabelEncoder()

In [112]:
Y = label_enc.transform(Y_raw)

In [114]:
import sklearn.model_selection as ms
import numpy as np

In [115]:
XTrain, XTest, YTrain, YTest = \
ms.train_test_split(X,Y, test_size=0.3, random_state = 1)

In [116]:
from sklearn.linear_model import LogisticRegression

In [117]:
pen_val = ['l1', 'l2']

In [118]:
C_val = 2. ** np.arange(-5, 10, step = 2)

In [119]:
grid_s = [{'C': C_val, 'penalty': pen_val}]

In [120]:
model = LogisticRegression()

In [121]:
cv_logr = GridSearchCV(estimator = model, param_grid = grid_s, \
                       cv = ms.KFold(n_splits = 10))

In [126]:
cv_logr.fit(XTrain, YTrain)

ValueError: could not convert string to float: '?'

In [78]:
best_c = cv_logr.best_params_['C']
best_penalty = cv_logr.best_params_['penalty']

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
# Extra: test how KNN, logistic, Naive Bayes and RF perform on same db?